In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

tf.keras.backend.clear_session()

In [2]:
from src.models import imagenet_ontop_model
from src import custom_losses, custom_metrics, optimizers
from src.data import data

Using TensorFlow backend.


In [10]:
batch_size = 128
n_classes = 6
epochs = 24
img_size = 224
n_channels = 3

In [11]:
model = imagenet_ontop_model.ImageNetOntop_Model(batch_size, n_classes, epochs, img_size, n_channels, version=7)

In [12]:
from src.data import data

In [13]:
paths = data.PATH()
dataset_path = f'{paths.PROCESSED_DATA_PATH}/'
dataset = 'vision_based_dataset'
test_dataset_path = f'{dataset_path}/{dataset}/'

In [14]:
train_generator, validation_generator, test_generator = model.get_image_data_generator(test_dataset_path, train=True, validation=True, test=True, class_mode_validation='categorical', class_mode_test='categorical')

Found 114361 images belonging to 6 classes.
Found 6305 images belonging to 6 classes.
Found 6328 images belonging to 6 classes.


In [15]:
weights = model.get_class_weights(train_generator.classes, model)
model.compile(loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)
# model.model.compile(optimizer='adam', loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)
# instance_model.compile(optimizer='adam', loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)

In [16]:
model.show_summary()

##### ImageNetOntop_Model #####
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D) 

In [17]:
model.fit_from_generator(path=f'{dataset_path}/{dataset}', 
                         train_generator=train_generator, validation_generator=validation_generator,
                         test_generator=test_generator,
                         evaluate_net=False, use_model_check_point=True, use_early_stop=False, weighted=True,
                         show_activations=False, n_workers=2)

W0112 23:27:32.455472 139808607541056 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.
W0112 23:27:32.456096 139808607541056 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification/src/models/base_model.py:354: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0112 23:27:32.458546 139808607541056 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification/src/models/base_model.py:355: The name tf.local_variables_initializer is deprecated. Please use tf.compat.v1.local_variables_initializer instead.

W0112 23:27:32.461135 139808607541056 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification/src/models/base_model.py:356: The name tf.keras.backend.get_session is deprecated. Please use tf.co

class_weights [2.09728947 0.87794411 1.08704042 0.8524606  0.87239869 0.87343812]
**** Class weights ****
[2.09728947 0.87794411 1.08704042 0.8524606  0.87239869 0.87343812]
*** ** *** *** *** ** ***


W0112 23:27:32.875103 139808607541056 deprecation.py:323] From /home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0112 23:27:35.381119 139808607541056 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/24
894/894 [==============================] - 797s 891ms/step - loss: 1.4901 - categorical_accuracy: 0.3517 - val_loss: 1.3397 - val_categorical_accuracy: 0.4220
Epoch 2/24
894/894 [==============================] - 926s 1s/step - loss: 1.3274 - categorical_accuracy: 0.4367 - val_loss: 1.2382 - val_categorical_accuracy: 0.4622
Epoch 3/24
894/894 [==============================] - 915s 1s/step - loss: 1.2634 - categorical_accuracy: 0.4744 - val_loss: 1.3602 - val_categorical_accuracy: 0.4882
Epoch 4/24
894/894 [==============================] - 915s 1s/step - loss: 1.2061 - categorical_accuracy: 0.5033 - val_loss: 1.3567 - val_categorical_accuracy: 0.5099
Epoch 5/24
894/894 [==============================] - 911s 1s/step - loss: 1.1580 - categorical_accuracy: 0.5261 - val_loss: 1.1274 - val_categorical_accuracy: 0.5289
Epoch 6/24
894/894 [==============================] - 891s 997ms/step - loss: 1.1251 - categorical_accuracy: 0.5437 - val_loss: 1.2462 - val_categorical_accuracy:

RuntimeError: Problems closing file (file write failed: time = Mon Jan 13 03:11:58 2020
, filename = '/home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification//models/ImageNetOntop_Model/2020-01-12__23_27//weights.15-1.05.hdf5', file descriptor = 104, errno = 28, error message = 'No space left on device', buf = 0x1cf94680, total write size = 2048, bytes this sub-write = 2048, bytes actually written = 18446744073709551615, offset = 4096)

In [18]:
model.model_path

'/home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification//models/ImageNetOntop_Model/2020-01-12__23_27/'

In [19]:
model.model.save_weights('/home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification//models/ImageNetOntop_Model/2020-01-12__23_27/weights.15-1.05.hdf5')

In [12]:
model.model_path

'/home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification//models/PlacesOntop_Model/2020-01-05__18_54/'

In [13]:
model = model.load_model(model.model_path)

In [11]:
model.model_is_trained = True

In [12]:
model.save_model()

weights [2.09728947 0.87794411 1.08704042 0.8524606  0.87239869 0.87343812]
